<a href="https://colab.research.google.com/github/jaycinguyen/Model-Fine-Tuning/blob/main/Model_Fine_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine-tune: A pretrained model like LLaMA is trained on a lot of general data (books, websites, etc.). It knows a lot, but:

- It doesn’t specialize in every possible task.
- Want the model to better understand your specific needs or data.

Fine-tuning allows us to make small adjustments so the model performs better for specific tasks.

In [ ]:
%%capture
#The package download is not displayed
!pip install unsloth
# Unsloth is open-source lib for efficient training and inference of LLMs
# Get the latest nightly Unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
# Maximum sequence length for input text the model can handle (2048 tokens)
dtype = None # Data type for model's parameters
# None for auto detection. Ex: Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True
# This flag indicates to load the model with 4-bit quantization.
# => Reduce memory usage, allows for faster loading. Can be False.

# => 4bit pre quantized models we support for 4x faster downloading + no OOMs (out of memory).
# List of pre-trained models available in the Unsloth lib support 4-bit quantization.
# maintaining performance.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # which pre-trained model to load
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

FastLanguageModel: A class from Unsloth to load and manage language models.

torch: open-source machine learning lib used for deep learning apps. For tensor data structure, handling input data (text converted into token IDs)


This part is to set up LoRA (Low-Rank Adaptation), a technique to adapt large models efficiently by updating only a subset of their parameters.

Because adjusting everything in the model will take a lot of time and memory

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    # Specify the existing model

    r=16,  # Choose any positive number! Recommended values include 8, 16, 32, 64, 128, etc.
    # Rank parameter for LoRA. The smaller this value, the fewer parameters will be modified.

    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
    # Specify the modules to which LoRA will be applied

    lora_alpha=16,
    # Alpha parameter for LoRA. This value determines the strength of the applied LoRA.

    lora_dropout=0,  # Currently, only supports dropout = 0
    # Dropout rate for LoRA. Currently supports only 0.
    # Dropout is a regularization technique used to prevent overfitting in neural networks. It works by randomly "dropping out" or deactivating some neurons (connections) during training.
    # Set to 0 if believe there's no overfit

    bias="none",  # Currently, only supports bias = "none"
    # Bias usage setting. Currently supports only the setting without bias.
    # Not use bias parameters in LoRA layers, probably to reduce complexity and memory usage.

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    # Whether to use gradient checkpointing to improve memory efficiency
    # Checkpointing is a technique to save memory during model training

    random_state = 3407,
    # Seed value for random number generation, , ensuring reproducibility of results
    # => With the same seed, the model will always initialize the same way and produce the same results, which is useful for experiments

    use_rslora = False,  # We support rank stabilized LoRA
    # RSLora is a modification of the standard LoRA technique that aims to stabilize the rank of the added low-rank matrices during training.
    # False because no oversit

    loftq_config = None, # And LoftQ
    # LoftQ is an optimazation technique or configuration specifically designed to enhance model performance
    # Set to None for simplicity
)


Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In LoRA, **"rank" (r)** refers to the size of the low-rank matrices used to approximate changes to the original model’s large matrices.

Instead of directly fine-tuning the full (large) weight matrices, LoRA decomposes the updates into two much smaller matrices of size 𝑑 × 𝑟 and 𝑟 × 𝑘, where r is the rank.


These are projection layers in the attention mechanism of transformer models.

- q_proj: Creates query vectors (used to find relevant information from other tokens).
- k_proj: Creates key vectors (used to align with queries).
- v_proj: Creates value vectors (information passed through attention).
- o_proj: Outputs the final transformed data from the attention block.

These are linear layers involved in feedforward networks within the transformer architecture.
- gate_proj: Controls how much information passes through (like a gate).
- up_proj / down_proj: Used for expanding and reducing dimensionality inside the model, often in the feedforward block.

In LoRA, the updated model weights are a combination of:

- Original model weights
- LoRA adjustment weights

**The formula looks something like this:**

New Weight = Original Weight + 𝛼/𝑟 × (LoRA Weights)

- r = Rank (size of the low-rank matrices in LoRA)
- alpha = Scaling factor (LoRA alpha)

By dividing by r, the LoRA weights are already small. Then multiplying by alpha scales them up/down to balance the effect.

- If the lora_alpha is too low, the adjustments will be too small to have impact, leading to ineffective fine-tuning
- If the lora_alpha is too high, the model lose original knowledge, leading to unstable


**What is a bias in neural networks?**
- A bias is like a constant added to the output of neurons in each layer.
- It helps the model shift the activation function’s curve, giving it more flexibility to fit the data or fine-tune outputs.

Normally, during training:

- The forward pass (input → output) calculates activations at each layer and stores them for the backward pass.
- During the backward pass, these stored activations are reused to calculate the gradients.

Problem:
Storing all these intermediate activations requires a lot of memory.

=> Instead of storing all intermediate activations during the forward pass, gradient checkpointing works like this:

- Save checkpoints at a few key layers (not all).
- When needed during backpropagation, recompute the missing activations on the fly from these checkpoints instead of storing them.

=> Save memory but trade off increasing computation time

# **Load dataset**

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN to the tokenized output!! Otherwise you'll get infinite generations!

def formatting_prompts_func(examples):
  instructions = examples["instruction"] #Retrieve "instruction" list from examples
  inputs = examples["input"]
  outputs = examples["output"]
  texts = []
  for instruction, input, output in zip(instructions, inputs, outputs):
    # zip() combines the three lists into tuples of corresponding elements, allowing  to process each example in a structured way.
    text = alpaca_prompt.format(instruction, input, output) #Create a "text" field that combine 3 sets in a template way
    # for each set of instruction, input, and output, the function formats a prompt string using the alpaca_prompt template.
    texts.append(text)
  return {"text" : texts,} #return a dictionary
pass

# Load dataset and map with formatting
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)



README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

# **Train the model**

In [ ]:
from trl import SFTTrainer
# SFTTrainer: specialized trainer for supervised fine-tuning of language models
from transformers import TrainingArguments
# TrainingArguments: A class that holds various arguments for training models in the Hugging Face Transformers library.
from unsloth import is_bfloat16_supported
# is_bfloat16_supported: A function from the unsloth library that checks whether the system supports the bfloat16 data type.

trainer = SFTTrainer(
    model = model, # model to fine-tune
    tokenizer = tokenizer, # tonkenizer convert text to appropriate input format for model
    train_dataset = dataset,
    dataset_text_field = "text",
    # specifies the key in the training dataset that contains the actual text data used for training the model.
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # Faster if num is larger, but use more memory as processes are independent
    # the number of parallel processes to use when applying functions to your dataset
    # Split the large dataset into 2 smaller, have 2 parallel process to go through
    packing = False, # Can make training 5x faster for short sequences.
    # Disables packing of sequences, which can speed up training when dealing with shorter sequences.

    # Define how the training process will be carried out
    args = TrainingArguments(
        per_device_train_batch_size = 2, # The number of examples processed together in a single forward/backward pass in each device (GPU or CPU)
        gradient_accumulation_steps = 4,
        # specifies how many gradient updates to accumulate before performing a backward pass and updating the model weights

        warmup_steps = 5,
        # defines the number of steps to perform learning rate warmup. During warmup, the learning rate gradually increases from 0 to the specified learning rate, which can help stabilize training in the early phases

        # num_train_epochs = 1, # Set this for 1 full training run.

        max_steps = 60,
        # total number of training steps to perform.
        # Once this number is reached, training will stop, regardless of how many epochs have been completed

        learning_rate = 2e-4, # The initial learning rate for the optimizer
        # Warmup: within 5 steps, the rate will increase to this target (2e-5)

        fp16 = not is_bfloat16_supported(), #Uses mixed precision training if bfloat16 is not supported. This can help save memory and improve speed
        bf16 = is_bfloat16_supported(), # If the system supports bfloat16, it will be used

        logging_steps = 1,
        # how frequently the model logs metrics or information (like loss, accuracy, or learning rate) during training
        # after every (1) step (i.e., after each batch is processed), the model will log training metrics

        optim = "adamw_8bit", ## Optimizer to use

        weight_decay = 0.01,
        # With weight_decay = 0.01, the model is slightly penalized for having large weights.
        # This helps prevent overfitting (but not much), especially on datasets where models are prone to memorize specific patterns.

        lr_scheduler_type = "linear", # how the learning rate changes during training
        seed = 3407,
        output_dir = "outputs",
        # The directory where training artifact (trained model, checkpoints, logs, and other related files) will be saved
        # a directory named outputs/ is automatically created (if it doesn’t already exist) during the training process
        report_to="none"
    ),
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


**Gradient**

- The gradient of the ***loss*** (difference between prediction and actual target data) function with respect to the model's parameters (weights) indicates the direction and rate of change of the loss function.
- It is a vector of partial derivatives, showing how much the loss would increase or decrease with small changes in the parameters.

=> Tell you how to adjust the parameters to reduce the loss.

Gradient update: adjust the model paremeters using calculated gradient.

=> Repeat this process for a specified number of iterations (epochs) or until the loss converges to a minimum

With large dataset, cannot repeat many times. => Gradient Accumulation (single update)

An ***epoch*** is one complete iteration over the entire training dataset. In other words, during one epoch, every sample in the training dataset is used once to update the model parameters.
Training Process:

When you set num_train_epochs = 1, it means the model will go through the entire dataset only once during the training process.

**bfloat16**
- Machine learning models often use floating-point arithmetic to represent numbers. Traditionally, FP32 (32-bit floating-point precision) is used.
- bfloat16 (16-bit brain floating-point) is a lower-precision format, primarily used to speed up training by using fewer bits, without sacrificing much accuracy compared to FP32.

**Overfitting**

- Overfitting happens when the model performs very well on the training data but poorly on unseen test data. This occurs when the model becomes too complex and "memorizes" the training data instead of learning general patterns.

- ***Weight decay*** works by shrinking the magnitude of the model’s weights slightly during each update. This prevents the weights from growing too large, forcing the model to find a simpler solution that can generalize better to unseen data.


# **Show current memory stats**

Advoid memory error

Ensure have enought memory to load the model and datasets

In [ ]:
gpu_stats = torch.cuda.get_device_properties(0)
# returns details like the name of the GPU, total memory, and other specs
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
# returns the maximum amount of GPU memory reserved by your program so far
# divide by 1024^3 to convert it to GB
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
#  total GPU memory available
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.004 GB of memory reserved.


**Initiates the training process**

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.590600
2,2.129700
3,1.677200
4,1.866200
5,1.684800
6,1.494400
7,1.082900
8,1.275300
9,1.150800
10,1.127400


Training loss:
- Measure of how well a machine learning model is performing during training.
- It quantifies the difference between the model's predictions and the actual target values (the ground truth) for the training data

# **Show final memory and time stats**

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

511.8706 seconds used for training.
8.53 minutes used for training.
Peak reserved memory = 8.0 GB.
Peak reserved memory for training = 1.996 GB.
Peak reserved memory % of max memory = 54.245 %.
Peak reserved memory for training % of max memory = 13.534 %.


# **Inference**

Run the model! Can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above

FastLanguageModel.for_inference(model) # Enable native 2x faster inference


# convert the formatted prompt into token IDs that the model can understand
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Give it in order from smaller to bigger", # instruction
        "14, 155, 22, 3, 55, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")
# "pt": output should be in the format suitable for PyTorch tensors.
# cuda: moves the input tensors to the GPU for faster processing

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
#Use_cache: model should use cached states from previous steps to speed up the generation process
#=> Instead of regenerating everything again + the previous output to create a new output


# THIS OUTPUT WILL NOT HAVE GOOD STRUCTURE
#outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
#tokenizer.batch_decode(outputs)


decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
# takes the output tokens produced by the model and converts them back into human-readable text

# Print each output in a more structured format
for output in decoded_outputs:
    print(output)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Give it in order from smaller to bigger

### Input:
14, 155, 22, 3, 55, 8

### Response:
The given numbers can be arranged in ascending order as follows: 3, 8, 14, 22, 55, 155.


***TextStreamer:*** see the generation token by token, instead of waiting the whole time

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418, 317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465, 14930352, 24157817, 39088169, 632459


# **Saving, loading finetuned models**
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

ONLY saves the LoRA adapters, and not the full model.

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

if want to load the LoRA adapters we just saved for inference, set False to True:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Suggest places to go?", # instruction
        "I'm in Cincinnati", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Suggest places to go?

### Input:
I'm in Cincinnati

### Response:
Cincinnati is a great city with plenty of options for things to do and places to go. Some suggestions include:

1. Visit the Cincinnati Museum Center at Union Terminal, which features exhibits on local history, natural history, and science.

2. Explore the Cincinnati Zoo and Botanical Garden, which is home to over 500 animal species and 3,000 plant species.

3. Take a walk or bike ride along the Ohio River or the Ohio River Trail, which offers scenic views of the city and the river.

4. Visit the Cincinnati Art Museum, which houses an extensive collection of art from around the world.

5. Check out


***Training:***

- Teach the model by feeding it data and correct answers.
- The goal is to make the model learn patterns so it can give good predictions for new inputs in the future.
- Training takes time and uses a lot of resources like GPU. After training, we save the trained model so we can use it later for inference without re-training.


***Inference:***

- Trained model generates predictions for new inputs (like answering questions or continuing sequences).
- Don’t change the model’s knowledge during inference.

=> Use what it learned from training to get outputs.
- Load the saved model to perform inference if the session is restarted or if you're only interested in generating predictions.

***When True:***
- Loads the model and tokenizer from the saved checkpoint (lora_model folder) into memory. This is needed after restarting sessions.
- Do this when after restart session or the model and tokenizer is not in memory

***When False:***
- Skips loading since the model and tokenizer are already in memory (i.e., they were defined earlier in the same session).

In [ ]:
# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")
